In [ ]:
# Mark Eilers
# CS 499: Senior Capstone
# Enhancement Artifact 3
# August 10, 2024

# Import required libraries for the web application
from jupyter_plotly_dash import JupyterDash
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

# Custom class import for managing database interactions (update module and class name as needed)
from shelter_db_manager import ShelterDBManager

##############################
# Data Handling and Preparation
##############################

# Database credentials and connection
user = "aacuser"
pwd = "cs340"
db_name = "AAC"
shelter_db = ShelterDBManager(user, pwd, db_name)

# Convert MongoDB query result to a pandas DataFrame for use in Dash
animal_df = pd.DataFrame.from_records(shelter_db.read({}))

#########################
# Dashboard Layout and View
#########################

# For testing in Jupyter Notebook:
# app = JupyterDash('Animal Rescue Dashboard')

# For regular execution in the terminal:
app = dash.Dash('Animal Rescue Dashboard')

# Encoding the logo image for display
logo_file = 'GraziosoSalvareLogo.png'
encoded_logo = base64.b64encode(open(logo_file, 'rb').read()).decode()

# Define the app layout
app.layout = html.Div([
    # Hidden div for storing intermediate data
    html.Div(id='hidden-div', style={'display': 'none'}),

    # Logo and header section
    html.Center([
        html.A([
            html.Img(id='logo-image',
                     src=f'data:image/png;base64,{encoded_logo}',
                     alt='Grazioso Salvare Logo',
                     style={'width': 225})
        ], href="https://www.arsari.us", target="_blank"),
        html.H1("Rescue Animal Search Dashboard"),
        html.H5("Developed by Mark Eilers", style={'color': 'green'})
    ]),
    html.Hr(),

    # Filter section with buttons and dropdown
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Span("Filter by:", style={'margin': 6}),
        html.Span(html.Button(id='cat-filter-btn', n_clicks=0, children='Cats'), style={'margin': 6}),
        html.Span(html.Button(id='dog-filter-btn', n_clicks=0, children='Dogs'), style={'margin': 6}),
        html.Span(html.Button(id='reset-filters', n_clicks=0, children='Reset', style={'background-color': 'red', 'color': 'white'}), style={'margin': 6}),
        html.Span("or", style={'margin': 6}),
        html.Span([
            dcc.Dropdown(
                id='category-filter',
                options=[
                    {'label': 'Water Rescue', 'value': 'wr'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'mwr'},
                    {'label': 'Disaster Rescue or Individual Tracking', 'value': 'drit'}
                ],
                placeholder="Select a Dog Category Filter",
                style={'marginLeft': 5, 'width': 350}
            )
        ])
    ]),
    html.Hr(),

    # Data table for displaying the list of animals
    dt.DataTable(
        id='animal-table',
        columns=[{"name": col, "id": col, "deletable": False, "selectable": True} for col in animal_df.columns],
        data=animal_df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),

    # Layout for chart and map side by side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='chart-div', className='col s12 m6'),
        html.Div(id='map-div', className='col s12 m6')
    ]),

    # Footer with project details
    html.Div([
        html.Hr(),
        html.P([
            "Module 5-2 Artifact Enhancement.",
            html.Br(),
            "CS-499 Senior Capstone, Southern New Hampshire University",
            html.Br(),
            "August 12, 2024"
        ], style={'fontSize': 12})
    ])
])

#############################################
# Interaction Between Components / Callbacks
#############################################

# Callback to update the data table based on selected filters
@app.callback(
    Output('animal-table', 'data'),
    [Input('category-filter', 'value'),
     Input('cat-filter-btn', 'n_clicks'),
     Input('dog-filter-btn', 'n_clicks')]
)
def filter_animals(selected_category, cats_btn, dogs_btn):
    # Determine the query based on the selected filters
    if selected_category == 'drit':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    elif selected_category == 'mwr':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif selected_category == 'wr':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif int(cats_btn) > int(dogs_btn):
        query = {"animal_type": "Cat"}
    elif int(dogs_btn) > int(cats_btn):
        query = {"animal_type": "Dog"}
    else:
        query = {}

    # Retrieve and return filtered data
    filtered_df = pd.DataFrame(list(shelter_db.read(query)))
    return filtered_df.to_dict('records')

# Callback to reset the cat and dog filter buttons
@app.callback(
    [Output('cat-filter-btn', 'n_clicks'),
     Output('dog-filter-btn', 'n_clicks')],
    [Input('reset-filters', 'n_clicks')]
)
def reset_filters(reset_btn):
    return 0, 0

# Callback to apply custom styling for selected columns or rows in the table
@app.callback(
    Output('animal-table', 'style_data_conditional'),
    [Input('animal-table', 'selected_columns'),
     Input('animal-table', "derived_viewport_selected_rows"),
     Input('animal-table', 'active_cell')]
)
def style_table(selected_columns, selected_rows, active_cell):
    # Highlight selected row or column
    if active_cell:
        highlight_style = [{'if': {'row_index': active_cell['row']}, 'background_color': '#a5d6a7'}]
    else:
        highlight_style = [{'if': {'row_index': i}, 'background_color': '#a5d6a7'} for i in selected_rows]

    return (highlight_style +
            [{'if': {'column_id': col}, 'background_color': '#80deea'} for col in selected_columns])

# Callback to generate pie chart based on table data
@app.callback(
    Output('chart-div', 'children'),
    [Input('animal-table', 'derived_viewport_data')]
)
def generate_pie_chart(view_data):
    df = pd.DataFrame(view_data)
    pie_chart = px.pie(df, names='breed', title='Animal Breeds Pie Chart')
    return [dcc.Graph(figure=pie_chart)]

# Callback to generate map based on selected row in the table
@app.callback(
    Output('map-div', 'children'),
    [Input('animal-table', 'derived_viewport_data'),
     Input('animal-table', 'derived_viewport_selected_rows'),
     Input('animal-table', 'active_cell')]
)
def update_map(view_data, selected_rows, active_cell):
    df = pd.DataFrame(view_data)

    # Determine the selected row
    if active_cell:
        selected_row = active_cell['row']
    else:
        selected_row = selected_rows[0]

    # Extract relevant details from the selected row
    lat = df.loc[selected_row, 'location_lat']
    lon = df.loc[selected_row, 'location_long']
    name = df.loc[selected_row, 'name']
    breed = df.loc[selected_row, 'breed']
    animal_type = df.loc[selected_row, 'animal_type']
    age = df.loc[selected_row, 'age_upon_outcome']

    if not name:
        name = "No Name"

    # Create and return the map component with a marker for the selected location
    map_figure = dl.Map(center=[lat, lon], zoom=13, children=[
        dl.TileLayer(),
        dl.Marker(position=[lat, lon], children=[
            dl.Popup([
                html.H4(f'{name} ({animal_type})'),
                html.P(f'Breed: {breed}'),
                html.P(f'Age: {age} weeks')
            ])
        ])
    ])

    return [dl.DashLeaflet(map_figure)]

if __name__ == '__main__':
    app.run_server(debug=True)
